<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done


 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=0a997f8c56c53b1b9f1ada729a85ff833e7c82b3a862ffa8ac0b59ec0abd73bb
  Stored in directory: /home/runner/.cache/pip/wheels/e3/3a/ee/4955a26c90a4b7deb6d725dc8ec7b8604a7aef44e43a2e8af7
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.32.0
    Uninstalling protobuf-6.32.0:
      Successfully uninstalled protobuf-6.32.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-09-03 13:43:56
-------------------
qualified stocks: 91
with latest results: 26
still star stocks: 15
-------------------
Initial Investment:  1.22 C
CY Investment:  1.49 C
Reserve:  3.82 L
Current:  1.40 C
-------------------
Today PnL: 58.46 K (0.42%)
Current PnL: -17.80 L (-11.97%)
CY Booked + Current PnL: -8.37 L (-5.63%)
-------------------
Total profit:  3.22 L
Total loss:  -21.02 L
-------------------
Total Booked + Current PnL: 18.61 L (15.29%)
Total Booked PnL: 36.41 L (29.92%)
Curr Year Booked PnL: 9.43 L (6.74%)
Prev Year Booked PnL: 26.98 L (22.17%)
Est FTT:  2.19 C
Est FTT PnL: 79.01 L (56.5%)
Deployed:  1.22 C
Current:  1.40 C
CAGR/XIRR %: 8.79%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
35,HINDALCO,756.01,-14.12,70.0,H-LC,8.66,118904.0,14592.0,2057.0,3.08,13.99,1.73,15.96,9.0,7.09,0.87,33.25,X5K,ATH,METALS
26,DIXON,18940.15,-29.21,69.0,H-MC,6.82,141664.0,26728.0,9860.0,0.88,23.25,6.96,31.83,81.0,2.71,1.04,46.84,X40N,ATH,IT
81,TTKPRESTIG,770.00,107.47,71.0,M-SC,6.97,89355.0,-11422.0,11518.0,0.02,-11.33,12.89,0.09,245.0,-0.99,0.66,17.39,OX40N,NTT,DURABLES
4,APOLLOHOSP,8285.00,-14.97,57.0,H-LC,8.93,169928.0,13751.0,12337.0,0.55,8.80,7.26,16.71,22.0,1.11,1.25,27.78,X40N,BTT,HEALTHCARE
66,SBIN,863.00,-13.25,49.0,M-LC,4.44,214637.0,13158.0,14059.0,0.75,6.53,6.55,13.51,60.0,0.94,1.58,19.95,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
49,KPIGREEN,731.35,5.18,45.0,H-SC,11.08,125282.0,-15.0,59020.0,1.04,-0.01,47.11,47.09,141.0,-0.00,0.92,56.80,MH,ATH,POWER
55,PGHH,18062.58,-30.02,56.0,H-MC,5.36,202470.0,1650.0,68475.0,-0.79,0.82,33.82,34.92,80.0,0.02,1.49,5.54,X40,ATH,FMCG


In [11]:
# top 5 to enter based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
47,JIOFIN,387.00,-1.39,40.0,H-LC,14.27,218085.0,679.0,52820.0,-0.06,0.31,24.22,24.61,48.0,0.01,1.60,55.30,XY24,BTT,FINANCE
33,HCLTECH,1943.91,0.22,41.0,H-LC,9.93,229267.0,-12657.0,75933.0,-0.37,-5.23,33.12,26.15,8.0,-0.17,1.69,8.34,X40,ATH,IT
60,RELIANCE,1533.00,-14.79,41.0,H-LC,7.65,213455.0,3089.0,25700.0,0.13,1.47,12.04,13.68,37.0,0.12,1.57,18.23,XY25,NTT,REFINERIES
78,TCS,4998.00,-27.08,48.0,H-LC,14.72,291127.0,-54829.0,178694.0,-0.46,-15.85,61.38,35.80,1.0,-0.31,2.14,3.13,X40,BTT,IT
51,LTIM,7230.20,-6.65,49.0,H-LC,2.38,231750.0,-14401.0,93604.0,-1.78,-5.85,40.39,32.18,16.0,-0.15,1.70,28.08,X200,ATH,IT


In [12]:
# top 5 to enter based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LTIM,7230.20,-6.65,49.0,H-LC,2.38,231750.0,-14401.0,93604.0,-1.78,-5.85,40.39,32.18,16.0,-0.15,1.70,28.08,X200,ATH,IT
54,NESTLEIND,1377.00,-9.24,60.0,H-LC,2.91,279185.0,13759.0,43022.0,-0.65,5.18,15.41,21.40,11.0,0.32,2.05,12.64,XY25,NTT,FMCG
45,ITC,452.00,-37.50,54.0,H-LC,3.25,201566.0,1428.0,19471.0,1.34,0.71,9.66,10.44,4.0,0.07,1.48,6.32,X40,NTT,FMCG
20,CIPLA,1795.00,-18.41,60.0,H-LC,5.11,217419.0,12919.0,28504.0,1.13,6.32,13.11,20.25,10.0,0.45,1.60,15.60,X40N,BTT,PHARMA
87,VBL,671.64,-10.86,52.0,H-LC,7.47,321042.0,5200.0,107453.0,0.80,1.65,33.47,35.67,5.0,0.05,2.36,15.73,X40N,ATH,FMCG


In [13]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
63,SAIL,228.00,40.34,60.0,M-MC,7.59,221484.0,-3478.0,171141.0,4.49,-1.55,77.27,74.53,192.0,-0.02,1.63,28.59,XY24,BTT,STEEL
59,RELAXO,1176.00,-30.49,64.0,H-SC,6.27,64506.0,-51032.0,86025.0,3.57,-44.17,133.36,30.28,136.0,-0.59,0.47,26.82,X40N,NTT,FOOTWEAR
53,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109790.0,9286.0,21354.0,3.30,9.24,19.45,30.49,79.0,0.43,0.81,44.64,MH,ATH,METALS
35,HINDALCO,756.01,-14.12,70.0,H-LC,8.66,118904.0,14592.0,2057.0,3.08,13.99,1.73,15.96,9.0,7.09,0.87,33.25,X5K,ATH,METALS
8,AWL,485.00,-64.43,59.0,H-MC,6.33,218831.0,-49616.0,181301.0,3.05,-18.48,82.85,49.05,116.0,-0.27,1.61,12.51,XY24,NTT,FMCG


In [14]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,HONAUT,58357.33,-21.77,39.0,H-SC,3.61,110265.0,-17673.0,64803.0,-2.40,-13.81,58.77,36.84,143.0,-0.27,0.81,12.88,X40N,ATH,ELECTRICAL
51,LTIM,7230.20,-6.65,49.0,H-LC,2.38,231750.0,-14401.0,93604.0,-1.78,-5.85,40.39,32.18,16.0,-0.15,1.70,28.08,X200,ATH,IT
67,SFL,1287.00,42.17,61.0,M-SC,8.96,187721.0,-74497.0,144320.0,-1.59,-28.41,76.88,26.63,229.0,-0.52,1.38,16.71,XY24,NTT,MISC
23,COLPAL,3767.14,1.82,59.0,H-MC,9.59,185297.0,-26444.0,108547.0,-1.58,-12.49,58.58,38.77,84.0,-0.24,1.36,10.29,XY25,ATH,FMCG
88,VOLTAS,1918.49,1.58,64.0,H-MC,4.99,213000.0,21258.0,74763.0,-1.43,11.09,35.10,50.08,99.0,0.28,1.57,18.64,XY25,ATH,AC


In [15]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
81,TTKPRESTIG,770.0,107.47,71.0,M-SC,6.97,89355.0,-11422.0,11518.0,0.02,-11.33,12.89,0.09,245.0,-0.99,0.66,17.39,OX40N,NTT,DURABLES
82,UJJIVANSFB,60.0,84.36,45.0,H-SC,13.82,116370.0,-26109.0,45629.0,0.07,-18.32,39.21,13.70,163.0,-0.57,0.86,37.35,OX40N,NTT,BANKS
48,KANSAINER,340.0,-68.34,54.0,H-SC,3.71,218745.0,-50922.0,87257.0,0.44,-18.88,39.89,13.47,138.0,-0.58,1.61,10.64,XY24,NTT,PAINTS
41,INDIGOPNTS,1408.0,115.40,51.0,M-SC,3.05,141360.0,-33199.0,33234.0,1.72,-19.02,23.51,0.02,221.0,-1.00,1.04,22.96,OX40N,NTT,PAINTS
19,CERA,9475.0,-20.29,43.0,H-SC,3.09,113580.0,-30223.0,56972.0,-0.36,-21.02,50.16,18.60,149.0,-0.53,0.84,25.34,OX40N,NTT,CERAMICS


In [16]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,TCS,4998.00,-27.08,48.0,H-LC,14.72,291127.0,-54829.0,178694.0,-0.46,-15.85,61.38,35.80,1.0,-0.31,2.14,3.13,X40,BTT,IT
43,INFY,2275.00,-18.16,50.0,H-LC,11.45,319223.0,6335.0,165358.0,-0.05,2.02,51.80,54.87,3.0,0.04,2.35,8.81,X40,BTT,IT
45,ITC,452.00,-37.50,54.0,H-LC,3.25,201566.0,1428.0,19471.0,1.34,0.71,9.66,10.44,4.0,0.07,1.48,6.32,X40,NTT,FMCG
87,VBL,671.64,-10.86,52.0,H-LC,7.47,321042.0,5200.0,107453.0,0.80,1.65,33.47,35.67,5.0,0.05,2.36,15.73,X40N,ATH,FMCG
1,ABB,7934.00,-40.45,49.0,H-LC,13.42,248899.0,-12720.0,131941.0,0.72,-4.86,53.01,45.57,7.0,-0.10,1.83,5.91,AR,NTT,ELECTRICAL


In [17]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,77.0,H-MC,19.42,186462.0,35549.0,21760.0,0.29,23.56,11.67,37.97,93.0,1.63,1.37,54.82,AR,ATH,AUTO
26,DIXON,18940.15,-29.21,69.0,H-MC,6.82,141664.0,26728.0,9860.0,0.88,23.25,6.96,31.83,81.0,2.71,1.04,46.84,X40N,ATH,IT
27,DMART,5201.00,-7.11,68.0,H-LC,14.35,232020.0,47539.0,22831.0,1.18,25.77,9.84,38.14,38.0,2.08,1.71,39.11,X40N,NTT,FMCG


In [18]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
14,BLUESTARCO,2326.38,13.61,66.0,H-MC,3.23,195980.0,31310.0,36648.0,1.00,19.01,18.70,41.28,89.0,0.85,1.44,28.51,X40N,ATH,AC
88,VOLTAS,1918.49,1.58,64.0,H-MC,4.99,213000.0,21258.0,74763.0,-1.43,11.09,35.10,50.08,99.0,0.28,1.57,18.64,XY25,ATH,AC
34,HEROMOTOCO,5949.07,0.81,77.0,H-MC,19.42,186462.0,35549.0,21760.0,0.29,23.56,11.67,37.97,93.0,1.63,1.37,54.82,AR,ATH,AUTO
24,DABUR,735.00,-2.58,64.0,H-MC,3.94,211906.0,14878.0,74739.0,-0.23,7.55,35.27,45.49,102.0,0.20,1.56,21.58,XY24,BTT,FMCG
18,CAMS,5211.76,-6.27,47.0,H-SC,5.62,106481.0,4477.0,39451.0,0.02,4.39,37.05,43.06,122.0,0.11,0.78,22.26,X40N,ATH,MISC


In [19]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
34,HEROMOTOCO,5949.07,0.81,77.0,H-MC,19.42,186462.0,35549.0,21760.0,0.29,23.56,11.67,37.97,93.0,1.63,1.37,54.82,AR,ATH,AUTO
53,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109790.0,9286.0,21354.0,3.30,9.24,19.45,30.49,79.0,0.43,0.81,44.64,MH,ATH,METALS
89,WHIRLPOOL,2270.00,-40.41,58.0,M-SC,4.91,100718.0,9220.0,69536.0,0.69,10.08,69.04,86.07,223.0,0.13,0.74,46.26,XR,NTT,DURABLES
40,INDIAMART,4850.92,-52.21,42.0,H-SC,12.48,133507.0,10171.0,123587.0,-1.14,8.25,92.57,108.45,119.0,0.08,0.98,32.44,AR,ATH,MISC
90,WIPRO,420.00,-11.98,50.0,M-LC,7.07,154907.0,3962.0,105492.0,-0.43,2.62,68.10,72.51,53.0,0.04,1.14,8.81,XR,NTT,IT


In [20]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
64,SAMMAANCAP,326.00,-172.73,69.0,M-SC,7.26,83556.0,-18654.0,112040.0,-0.07,-18.25,134.09,91.37,208.0,-0.17,0.61,35.64,XY25,NTT,FINANCE
3,ALKYLAMINE,4546.37,-2.66,49.0,H-SC,16.22,90562.0,-10401.0,104934.0,2.15,-10.30,115.87,93.63,148.0,-0.10,0.67,34.89,SR,ATH,CHEMICALS
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.31,92141.0,-21408.0,61458.0,0.65,-18.85,66.70,35.27,268.0,-0.35,0.68,125.52,XR,NTT,HOTELS
52,MASFIN,398.61,-17.34,55.0,H-SC,13.26,94140.0,-3840.0,25446.0,2.23,-3.92,27.03,22.05,152.0,-0.15,0.69,36.84,XR,ATH,FINANCE
53,NATIONALUM,247.44,-45.32,71.0,H-MC,0.47,109790.0,9286.0,21354.0,3.30,9.24,19.45,30.49,79.0,0.43,0.81,44.64,MH,ATH,METALS


In [21]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Dev%_PE,RSI_14,Conviction,Spread%,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
22,COFFEEDAY,80.00,-37.63,68.0,L-SC,33.31,92141.0,-21408.0,61458.0,0.65,-18.85,66.70,35.27,268.0,-0.35,0.68,125.52,XR,NTT,HOTELS
34,HEROMOTOCO,5949.07,0.81,77.0,H-MC,19.42,186462.0,35549.0,21760.0,0.29,23.56,11.67,37.97,93.0,1.63,1.37,54.82,AR,ATH,AUTO
59,RELAXO,1176.00,-30.49,64.0,H-SC,6.27,64506.0,-51032.0,86025.0,3.57,-44.17,133.36,30.28,136.0,-0.59,0.47,26.82,X40N,NTT,FOOTWEAR
27,DMART,5201.00,-7.11,68.0,H-LC,14.35,232020.0,47539.0,22831.0,1.18,25.77,9.84,38.14,38.0,2.08,1.71,39.11,X40N,NTT,FMCG
14,BLUESTARCO,2326.38,13.61,66.0,H-MC,3.23,195980.0,31310.0,36648.0,1.00,19.01,18.70,41.28,89.0,0.85,1.44,28.51,X40N,ATH,AC


In [22]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.47
1,25,43.03
2,50,72.76


In [23]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
LC    38.01
SC    36.85
MC    25.14
Name: CurrAlloc%, dtype: float64

In [24]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.95
X40      15.30
X40N     14.12
XY25     11.36
AR        9.26
XR        8.90
OX40N     7.31
X5K       2.29
MH        1.73
X200      1.70
SR        1.08
Name: CurrAlloc%, dtype: float64

In [25]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-LC    29.89
H-MC    22.20
H-SC    22.05
M-SC    13.22
M-LC     7.06
M-MC     2.62
L-SC     1.58
L-LC     1.06
L-MC     0.32
Name: CurrAlloc%, dtype: float64

In [26]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.18,1.11,31.10
IT,13.17,-15.20,72.33
FINANCE,7.23,-22.31,67.41
BANKS,7.23,-16.22,65.57
PAINTS,5.91,-14.87,40.58
MISC,5.76,-17.37,84.64
ELECTRICAL,5.61,-7.76,67.15
AUTO,4.55,-9.30,56.85
AC,3.73,4.47,27.88


In [27]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2802147.0,22
AR,1217733.0,10
XR,1082411.0,13
X40,887593.0,10
X40N,512063.0,11
OX40N,466835.0,10
XY25,434554.0,8
SR,196761.0,2
X5K,126629.0,2


In [28]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2774119.0,27
M-SC,1449559.0,17
H-LC,1403515.0,18
H-MC,1185895.0,17
M-LC,419539.0,5
M-MC,322307.0,2
L-SC,246542.0,3
L-MC,59181.0,1
L-LC,40047.0,1


In [29]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,,sum,count
Conviction,Criteria,,
H-SC,XY24,933310.0,6
M-SC,XY24,861146.0,7
H-SC,AR,666458.0,5
H-LC,X40,588910.0,5
H-SC,XR,523701.0,6
H-MC,XY24,457148.0,4
H-LC,AR,369608.0,2
M-MC,XY24,322307.0,2
H-MC,X40,221007.0,4


In [30]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 31.0 seconds
